## Building The Model
-----

### Setup:
---

In [47]:
# Import dependencies
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [48]:
# Import the Combined Data from /Wrangling_Data_Exports
wrangled_df = pd.read_csv("Wrangling_Data_Exports/Wrangled_Data.csv", index_col=0)

# Display Dataframe
wrangled_df.head()

,Year,Month,Day,Time (EST),Season Week,Venue,City,State,Weather Condition,Temperature (F),...,Home Team Pre-Game Season W/L Ratio,Home Team Pre-Game Season Avg Points For,Home Team Pre-Game Season Avg Points Against,Outcome,Away Team,Away Team Pre-Game Season W Streak,Away Team Pre-Game Season L Streak,Away Team Pre-Game Season W/L Ratio,Away Team Pre-Game Season Avg Points For,Away Team Pre-Game Season Avg Points Against
0,2022.0,September,25,14:25,3.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,0.50,25.00,33.50,2.0,Los Angeles Rams,1.0,0.0,0.50,20.50,29.00
1,2022.0,September,25,14:25,3.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,0.50,25.00,33.50,2.0,Los Angeles Rams,1.0,0.0,0.50,20.50,29.00
2,2022.0,November,6,16:05,9.0,State Farm Stadium,Glendale,Arizona,Sunny,76.0,...,0.38,22.75,26.25,2.0,Seattle Seahawks,3.0,0.0,0.62,26.25,24.88
3,2022.0,November,6,16:05,9.0,State Farm Stadium,Glendale,Arizona,Sunny,76.0,...,0.38,22.75,26.25,2.0,Seattle Seahawks,3.0,0.0,0.62,26.25,24.88
4,2022.0,November,27,16:05,12.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,0.36,21.82,26.91,2.0,Los Angeles Chargers,0.0,2.0,0.50,22.70,25.80


### Prepare the Data:
---

In [49]:
# Display Dataframe
wrangled_df.head()

,Year,Month,Day,Time (EST),Season Week,Venue,City,State,Weather Condition,Temperature (F),...,Home Team Pre-Game Season W/L Ratio,Home Team Pre-Game Season Avg Points For,Home Team Pre-Game Season Avg Points Against,Outcome,Away Team,Away Team Pre-Game Season W Streak,Away Team Pre-Game Season L Streak,Away Team Pre-Game Season W/L Ratio,Away Team Pre-Game Season Avg Points For,Away Team Pre-Game Season Avg Points Against
0,2022.0,September,25,14:25,3.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,0.50,25.00,33.50,2.0,Los Angeles Rams,1.0,0.0,0.50,20.50,29.00
1,2022.0,September,25,14:25,3.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,0.50,25.00,33.50,2.0,Los Angeles Rams,1.0,0.0,0.50,20.50,29.00
2,2022.0,November,6,16:05,9.0,State Farm Stadium,Glendale,Arizona,Sunny,76.0,...,0.38,22.75,26.25,2.0,Seattle Seahawks,3.0,0.0,0.62,26.25,24.88
3,2022.0,November,6,16:05,9.0,State Farm Stadium,Glendale,Arizona,Sunny,76.0,...,0.38,22.75,26.25,2.0,Seattle Seahawks,3.0,0.0,0.62,26.25,24.88
4,2022.0,November,27,16:05,12.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,0.36,21.82,26.91,2.0,Los Angeles Chargers,0.0,2.0,0.50,22.70,25.80


In [50]:
# Create Target Array
target_array= wrangled_df["Outcome"].astype(int).values

# Display Array
target_array

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 3, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2,
       2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2,
       1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 3, 2, 1,
       2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2,
       1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 2,
       2, 1, 2, 2, 1, 2, 1, 2, 1])

In [46]:
# Create Features Dataframe
features_df = wrangled_df.drop(columns=["Outcome"])

# Display Dataframe
features_df

,Year,Month,Day,Time (EST),Season Week,Venue,City,State,Weather Condition,Temperature (F),...,Home Team Pre-Game Season L Streak,Home Team Pre-Game Season W/L Ratio,Home Team Pre-Game Season Avg Points For,Home Team Pre-Game Season Avg Points Against,Away Team,Away Team Pre-Game Season W Streak,Away Team Pre-Game Season L Streak,Away Team Pre-Game Season W/L Ratio,Away Team Pre-Game Season Avg Points For,Away Team Pre-Game Season Avg Points Against
0,2022.0,September,25,14:25,3.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,0.0,0.50,25.00,33.50,Los Angeles Rams,1.0,0.0,0.50,20.50,29.00
1,2022.0,September,25,14:25,3.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,0.0,0.50,25.00,33.50,Los Angeles Rams,1.0,0.0,0.50,20.50,29.00
2,2022.0,November,6,16:05,9.0,State Farm Stadium,Glendale,Arizona,Sunny,76.0,...,1.0,0.38,22.75,26.25,Seattle Seahawks,3.0,0.0,0.62,26.25,24.88
3,2022.0,November,6,16:05,9.0,State Farm Stadium,Glendale,Arizona,Sunny,76.0,...,1.0,0.38,22.75,26.25,Seattle Seahawks,3.0,0.0,0.62,26.25,24.88
4,2022.0,November,27,16:05,12.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),72.0,...,1.0,0.36,21.82,26.91,Los Angeles Chargers,0.0,2.0,0.50,22.70,25.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,2022.0,October,23,12:00,7.0,FedExField,Landover,Maryland,Cloudy,62.0,...,0.0,0.33,17.00,22.50,Green Bay Packers,0.0,2.0,0.50,17.83,20.50
225,2022.0,November,6,13:00,9.0,FedExField,Landover,Maryland,Cloudy,76.0,...,0.0,0.50,17.75,21.50,Minnesota Vikings,5.0,0.0,0.86,24.71,20.57
226,2022.0,November,27,13:00,12.0,FedExField,Landover,Maryland,Rain,49.0,...,0.0,0.55,19.45,20.27,Atlanta Falcons,1.0,0.0,0.45,23.55,24.91
227,2022.0,January,1,13:00,17.0,FedExField,Landover,Maryland,Sunny,62.0,...,2.0,0.47,19.00,20.87,Cleveland Browns,0.0,1.0,0.40,21.53,22.87


In [51]:
# Git Dummies
dummies_features_df = pd.get_dummies(features_df)

# Display Dataframe
dummies_features_df

,Year,Day,Season Week,Temperature (F),Home Team Pre-Game Season W Streak,Home Team Pre-Game Season L Streak,Home Team Pre-Game Season W/L Ratio,Home Team Pre-Game Season Avg Points For,Home Team Pre-Game Season Avg Points Against,Away Team Pre-Game Season W Streak,...,Away Team_New Orleans Saints,Away Team_New York Giants,Away Team_New York Jets,Away Team_Philadelphia Eagles,Away Team_Pittsburgh Steelers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Commanders
0,2022.0,25,3.0,72.0,1.0,0.0,0.50,25.00,33.50,1.0,...,0,0,0,0,0,0,0,0,0,0
1,2022.0,25,3.0,72.0,1.0,0.0,0.50,25.00,33.50,1.0,...,0,0,0,0,0,0,0,0,0,0
2,2022.0,6,9.0,76.0,0.0,1.0,0.38,22.75,26.25,3.0,...,0,0,0,0,0,0,1,0,0,0
3,2022.0,6,9.0,76.0,0.0,1.0,0.38,22.75,26.25,3.0,...,0,0,0,0,0,0,1,0,0,0
4,2022.0,27,12.0,72.0,0.0,1.0,0.36,21.82,26.91,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,2022.0,23,7.0,62.0,1.0,0.0,0.33,17.00,22.50,0.0,...,0,0,0,0,0,0,0,0,0,0
225,2022.0,6,9.0,76.0,3.0,0.0,0.50,17.75,21.50,5.0,...,0,0,0,0,0,0,0,0,0,0
226,2022.0,27,12.0,49.0,2.0,0.0,0.55,19.45,20.27,1.0,...,0,0,0,0,0,0,0,0,0,0
227,2022.0,1,17.0,62.0,0.0,2.0,0.47,19.00,20.87,0.0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# Create an Array from the Features Dataframe with Dummies
features_array = dummies_features_df.values

features_array

array([[2.022e+03, 2.500e+01, 3.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.022e+03, 2.500e+01, 3.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.022e+03, 6.000e+00, 9.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.022e+03, 2.700e+01, 1.200e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.022e+03, 1.000e+00, 1.700e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.022e+03, 8.000e+00, 1.800e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [54]:
# Scale the Data

# Create a Scaler
scaler = StandardScaler()

# Fit the StandardScaler
features_scaler = scaler.fit(features_array)

# Scale the Features
scaled_features = features_scaler.transform(features_array)

scaled_features

array([[ 0.        ,  1.13834758, -1.20482942, ..., -0.1902606 ,
        -0.1902606 , -0.16402997],
       [ 0.        ,  1.13834758, -1.20482942, ..., -0.1902606 ,
        -0.1902606 , -0.16402997],
       [ 0.        , -1.02451282, -0.05840586, ..., -0.1902606 ,
        -0.1902606 , -0.16402997],
       ...,
       [ 0.        ,  1.3660171 ,  0.51480592, ..., -0.1902606 ,
        -0.1902606 , -0.16402997],
       [ 0.        , -1.59368662,  1.47015889, ..., -0.1902606 ,
        -0.1902606 , -0.16402997],
       [ 0.        , -0.79684331,  1.66122949, ..., -0.1902606 ,
        -0.1902606 , -0.16402997]])

In [ ]:
# Primary Component Analysis?

In [ ]:
# Train/Test Split The Data

### Prepare the Model:
---

In [ ]:
# Create The Model/Models

### Train the Model:
---

In [ ]:
# Train the Model/Models

### Test the Model:
---

In [ ]:
# Test the Model/Models

### Analyze the Accuracy of the Model:
---

### Export the Trained Model:
---

In [ ]:
# Export the Trained Model to ../04-Creating_Applicaton/Building_Model_Exports
